In [1]:
import os
import importlib
from utility import Pi_util
from natsort import natsorted
import numpy as np
import pickle
import sys
import pandas as pd

path_to_lib: str = "/Users/zacharykelly/Documents/MATLAB/projects/combiExperiments/code/lightLogger/miniSpect"
sys.path.append(path_to_lib)
import MS_util
import importlib

In [2]:
experiment_path: str = "/Volumes/EXTERNAL1/queue30mins_0.1hz_0NDF"
use_mean_frame: bool = True

In [ ]:
importlib.reload(Pi_util)
importlib.reload(utility)

In [3]:
# First, define some helper functions
"""Parser for the raw World data per chunk"""
def world_parser(val_tuple: tuple) -> dict:
    print(f'Length of world vals: {len(val_tuple)}')

    # First value is always the frame buffer for this chunk 
    frame_buffer: np.ndarray = val_tuple[0].astype(np.uint8)
    
    # If we want to only use the mean of each frame, not the entire frame
    if(use_mean_frame):
        frame_buffer = np.mean(frame_buffer, axis=(2,3))

    # Flatten the frame buffer into one chunks worth of frames, instead of per second 
    frame_buffer = frame_buffer.reshape((frame_buffer.shape[0] * frame_buffer.shape[1], *frame_buffer.shape[2:]))

    # Second value is always the settings buffer for this chunk
    # The settings are in the format [duration, FPS gain, exposure] TODO: The FPS dimension does not currently exist, but going to add it
    settings_buffer: np.ndarray = val_tuple[1].astype(np.float64)

    # Third and Fourth values are always the num_captured_frames and observed FPS 
    num_captured_frames, observed_fps = val_tuple[2:]

    print(f'Frame Buffer Shape: {frame_buffer.shape}')
    print(f'Captured Frames: {num_captured_frames} | FPS: {observed_fps}')
                                                # Make this a float for MATLAB use later
    return {'frame_buffer': frame_buffer, 'settings_buffer': settings_buffer, 'num_frames_captured': float(num_captured_frames), 'FPS': observed_fps}

"""Parser for the raw Pupil data per chunk"""
def pupil_parser(val_tuple: tuple) -> dict:
    print(f'Length of Pupil vals: {len(val_tuple)}')

    # First value is always the frame buffer for this chunk
    frame_buffer: np.ndarray = val_tuple[0].astype(np.uint8)
    
    # If we want to only use the mean of each frame, not the entire frame
    if(use_mean_frame):
        frame_buffer = np.mean(frame_buffer, axis=(2,3))

    # Flatten the frame buffer into one chunks worth of frames, instead of per second 
    frame_buffer = frame_buffer.reshape((frame_buffer.shape[0] * frame_buffer.shape[1], *frame_buffer.shape[2:]))

    # Second value and third value are always num_captured_frames and observed FPS
    num_captured_frames, observed_fps = val_tuple[1:]

    print(f'Frame Buffer Shape: {frame_buffer.shape}')
    print(f'Captured Frames: {num_captured_frames} | FPS: {observed_fps}')
                                            # Make this a float for MATLAB use later
    return {'frame_buffer': frame_buffer, 'num_frames_captured': float(num_captured_frames), 'FPS': observed_fps}

In [4]:
importlib.reload(Pi_util)

# Define a dictionary of sensor initials and their respective parsers 
sensor_parsers: dict = {'W': world_parser, 
                        'P': pupil_parser,
                        'M': Pi_util.ms_parser}

# First, we must find the gather the sorted paths to the chunks
# which are stored in .pkl files
chunk_paths: list = natsorted([os.path.join(experiment_path, file) 
                                for file in os.listdir(experiment_path)
                                if '.pkl' in file])

# Initialize a list to hold the sorted chunks after 
# they have been loaded in
sorted_chunks: list = []

# Next, we will iterate over the chunk files and load them in 
for chunk_num, path in enumerate(chunk_paths):
    if(chunk_num > 1): continue

    print(f'Loading chunk: {chunk_num+1}/{len(chunk_paths)}')

    # Read in the file and append it to the sorted chunks 
    # list
    with open(path, 'rb') as f:
        # Read in the dictionary of values from this chunk
        chunk_dict: dict = pickle.load(f)

        # Append it to the sorted chunk list 
        sorted_chunks.append(chunk_dict)

Loading chunk: 1/180
Loading chunk: 2/180


/var/folders/fb/55l_tk897yjbwpmc6_mm1ylm0000gp/T/ipykernel_12060/2860891706.py:28: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  chunk_dict: dict = pickle.load(f)


In [5]:
importlib.reload(Pi_util)

# Next, we will iterate over the chunks and their sensors and their respective data in the chunk and parse them 
parsed_chunks: list = []
for chunk_num, chunk in enumerate(sorted_chunks):
    print(f'Parsing chunk: {chunk_num+1}/{len(sorted_chunks)}')
    # initialize a new dictionary to hold sensors' parsed information
    parsed_chunk: dict = {}

    for key, val in chunk.items():
        # Parse this sensor's data with its appropriate sensor
        parsed_data: dict = sensor_parsers[key](val)

        # Note this sensor's parsed info for this chunk 
        parsed_chunk[key] = parsed_data

    # Append this parsed chunk to the growing list of parsed chunks 
    parsed_chunks.append(parsed_chunk)

Parsing chunk: 1/2
Length of world vals: 4
Frame Buffer Shape: (2000,)
Captured Frames: 2006 | FPS: 200.50629854303023
Length of MS Vals: 3
BUFFER TYPE: <class 'numpy.ndarray'>
LS CHANNELS: int16 | shape: (10, 60)
BUFFER SIZE: 10
NUM ACCELERATION COLS: 30
NUM ANGULAR COLS: 
MEASUREMENT SHAPE: (100,)
MEASUREMENT SHAPE: (100,)
MEASUREMENT SHAPE: (100,)
MEASUREMENT SHAPE: (100,)
MEASUREMENT SHAPE: (100,)
MEASUREMENT SHAPE: (100,)
(10, 10)
(10, 2)
(100, 6)
Reading Buffer Shape: (10, 150)
Captured Frames: 10 | FPS: 0.9652881621649425


TypeError: unhashable type: 'DataFrame'

In [ ]:
importlib.reload(Pi_util)
parsed = Pi_util.parse_chunks_pkl(experiment_path, use_mean_frame=True)

print(parsed[0].keys())